In [ ]:
# DONE
# ine-irpe
# ine-cape
# ine-sin
# ine-rpe
# ine-drape
# ine-pa-ape
# ine-scape
# ine-jape
# ine-ape
# flw-ff
# flw-irpe
# flw-cape
# flw-rpe
# flw-sin
# flw-drape
# flw-cls-ape
# flw-pa-ape
# flw-scape
# flw-jape
# flw-ape
# eurosat-rpe
# eurosat-sin
# eurosat-ape
# eurosat-pa-ape
# eurosat-jape
# eurosat-cls-ape
# eurosat-scape
# svhn-rpe
# svhn-sin
# svhn-cape
# svhn-cls-ape
# svhn-drape
# svhn-scape
# svhn-pa-ape
# svhn-jape
# svhn-ape
# c10-rpe
# c10-pa-ape
# c10-scape
# c10-jape
# c10-wdape-v1
# c10-sin
# c10-opt-ape
# c10-cls-ape
# c10-opt-scape
# c10-opt-pa-ape
# c10-opt-jape

# TODO
# ine-ff
# ine-rope
# flw-rope
# eurosat-rope
# eurosat-ff
# eurosat-irpe
# eurosat-cape
# c10-rope
# c10-irpe
# flw-gl1-lrm1
# flw-gl1
# flw-gl1-lrm2
# eurosat-gl1-lrm1
# eurosat-gl1
# eurosat-gl1-lrm2

# TODO
# ('not found', 'eurosat-cls-ape', 2)
# ('not found', 'eurosat-cls-ape', 1)
# ('not found', 'eurosat-cls-ape', 0)
# ('not found', 'svhn-cape', 2)
# ('not found', 'svhn-cape', 1)
# ('not found', 'svhn-cape', 0)
# ('not found', 'c10-opt-scape', 2)
# ('not found', 'c10-opt-scape', 1)
# ('not found', 'c10-opt-scape', 0)

In [38]:
import wandb
import os
import numpy as np
import pandas as pd

from statistics import stats

run_names = [
    'ine-goog224-0',
    'ine-goog224-in-0',
]

def get_wandb_runs(run_name):
    api = wandb.Api()

    runs = api.runs(path="tudcv/vit-visualization", filters={
        "display_name": run_name,
        "tags": {"$nin": ['hidden', 'archived']},
    })
    print(f"{run_name}: {len(runs)} runs.")
    return runs

def load_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

def load_batched_pickles(path):
    import pickle
    import glob
    batch_files = sorted(glob.glob(os.path.join(path, 'batch_*.pkl')))
    shap_dfs = []
    for batch_file in batch_files:
        with open(batch_file, 'rb') as f:
            data = pickle.load(f)
            shap_dfs.append(data['batch_shap_df'])
    shap_df = pd.concat(shap_dfs, ignore_index=True)
    return shap_df

stats_to_update = [
    'misc/bias_class_correct',
    'values/mean_correct',
    'values/std_correct',
    'values/mean_correct_filtered',
    'values/std_correct_filtered',
    'values/mean_normed_correct',
    'values/std_normed_correct',
    'values/mean_normed_correct_filtered',
    'values/std_normed_correct_filtered'
]

todo = []
for name in run_names:
    runs = get_wandb_runs(name)
    for run in runs:
        print(f"Run [{run.id}] {run.name} (seed {run.config['seed']})")
        pickle_dir = f"{run.name}_seed{run.config['seed']}"
        pickle_path = os.path.join('results', pickle_dir)
        full_path = os.path.join(pickle_path, 'full.pkl')
        batched_path = os.path.join(pickle_path, 'batch_00000.pkl')
        if os.path.exists(full_path):
            data = load_pickle(full_path)
            shap_df = data['shap_df']
            old_stats = data['results']
            print('\tfull exists, {} rows'.format(len(shap_df)))
            new_stats = stats(shap_df)

            for key in old_stats.keys():
                if key in new_stats:
                    assert np.isclose(new_stats[key], old_stats[key]), f"Stats mismatch for {key}: {new_stats[key]} != {old_stats[key]}"
                    assert np.isclose(new_stats[key], run.summary['p-shap/' + key]), f"Stats mismatch for {key}: {new_stats[key]} != {run.summary[key]}"

            for key in new_stats.keys():
                if key not in old_stats:
                    old_stats['p-shap/' + key] = new_stats[key]

            run.summary.update(old_stats)

            # # Print the stats
            # print("Stats:")
            # for key, value in old_stats.items():
            #     print(f"{key}: {value}")
        elif os.path.exists(batched_path):
            # todo.append(('batched', run.name, run.config['seed']))
            shap_df = load_batched_pickles(pickle_path)
            print('\tbatched exists, {} rows'.format(len(shap_df)))
            new_stats = stats(shap_df)

            for key in new_stats.keys():
                if key not in stats_to_update:
                    assert np.isclose(new_stats[key], run.summary['p-shap/' + key]), f"Stats mismatch for {key}: {new_stats[key]} != {run.summary['p-shap/' + key]}"

            for key in new_stats.keys():
                if key in stats_to_update:
                    run.summary['p-shap/' + key] = new_stats[key]

            # Print the stats
            print("Stats:")
            for key, value in run.summary.items():
                print(f"{key}: {value}")
        else:
            print('\tno pickle found')
            todo.append(('not found', run.name, run.config['seed']))
            continue



        # run.update()
    print("\n")

print("TODO:")
for item in todo:
    print(item)

ine-goog224-0: 1 runs.
Run [8skmhuty] ine-goog224-0 (seed 0)
	batched exists, 416 rows


AssertionError: Stats mismatch for values/mean: 0.4537106126755624 != 0.4553321088885074